In [1]:
try:
    %load_ext cudf.pandas
    import pandas as pd

    print("CUDA acceleration is enabled.")
except ImportError:
    ("Running without CUDA acceleration")


In [5]:
from dotenv import dotenv_values, load_dotenv

from scipy.spatial.distance import pdist, squareform
from itertools import combinations
from collections import Counter
import numpy as np
from sklearn.cluster import KMeans
from sqlalchemy import create_engine, text
import polars as pl
import polars.selectors as cs
import os

import matplotlib.pyplot as plt
import polars as pl
import seaborn as sns
import pyarrow as pa
import pandas as pd
import plotly.express as px
import psycopg

# from download_from_S3 import get_data_s3

In [2]:
import warnings

# Suppress the specific warning about GPU-accelerated mergesort
warnings.filterwarnings(
    "ignore", message="GPU-accelerated mergesort is currently not supported"
)
pd.set_option("display.float_format", lambda x: "{:,.2f}".format(x))
pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', 20)
pd.set_option("display.width", None)


# Set global configuration for number formatting
pl.Config.set_fmt_str_lengths(100)  # For longer string display
pl.Config.set_tbl_formatting("UTF8_FULL_CONDENSED")
pl.Config.set_tbl_cell_numeric_alignment("RIGHT")
pl.Config.set_tbl_rows(20)
pl.Config.set_thousands_separator(",")  # Set thousands separator
pl.Config.set_float_precision(2)  # Set decimal places

polars.config.Config

In [3]:
%load_ext autoreload
%autoreload 2


In [4]:
# load_dotenv("instance.env")
# connection_string = f"postgresql://{os.getenv('user')}:{os.getenv('password')}@{os.getenv('host')}:{os.getenv('port')}/{os.getenv('database')}"
# conn = psycopg.connect(connection_string)


In [5]:
# def query_to_polars(conn, query):
#     df = pl.read_database(query, connection=conn)
#     return df

In [1]:
import numpy as np

def cumulative_mean_2d(A):
    """  
    A: np.array[num_row, num_column]        --- матрица-аргумент
    return S: np.array[num_row, num_column] --- выходная матрица кумулятивных сумм

    Функция принимает на вход матрицу A размерностью n x m и возвращает 
    матрицу с той же размерностью n x m, i-ая строчка которой есть последовательность 
    кумулятивных сумм элементов i-ой строки матрицы A
    """
    # Вычисляем кумулятивные суммы по строкам
    cumsum = np.cumsum(A, axis=1)
    
    # Создаем массив индексов от 1 до количества столбцов 
    indices = np.arange(1, A.shape[1] + 1)
    
    # Делим каждый элемент на его позицию для получения среднего
    result = cumsum / indices
    
    return result

In [8]:
X = np.array([[1, 2, 3],
              [4, 5, 6]])
print(cumulative_mean_2d(X))

[[1.  1.5 2. ]
 [4.  4.5 5. ]]


In [9]:
X

array([[1, 2, 3],
       [4, 5, 6]])

In [15]:
a = np.cumsum(X, axis = 1)

In [16]:
a

array([[ 1,  3,  6],
       [ 4,  9, 15]])

In [13]:
b = np.arange(1, X.shape[1] + 1)

In [17]:
a/b

array([[1. , 1.5, 2. ],
       [4. , 4.5, 5. ]])

In [32]:
import numpy as np

def transform(X):
    """
    Transform array X according to rules:
    1. Replace elements with odd indices with 1
    2. Cube elements with even indices
    3. Reverse the array
    
    Parameters:
        X: np.array - input array
        
    Returns:
        np.array - transformed array
    """
    Y = X.copy()
    Y[1::2] = 1
    Y[0::2] = Y[0::2] ** 3
    # Y = Y[::-1]
    return Y

# Пример работы функции
X = np.array([3, 1, 4, -1, 2, 3, 0])
Y = transform(X)
print(Y)  # array([27, 1, 64, 1, 8, 1, 0])

[27  1 64  1  8  1  0]


In [ ]:
X = np.array([3, 1, 4, -1, 2, 3, 0])

Y = transform(X)


In [30]:
Y

array([27,  1, 64,  1,  8,  1,  0])

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Загрузка данных
url = "https://stepik.org/media/attachments/lesson/779915/fs_task1_10f.csv"
data = pd.read_csv(url)

# Разделение данных на признаки и целевую переменную
X = data.drop('target', axis=1)
y = data['target']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Получение важности признаков
importances = model.feature_importances_

# Создание DataFrame для удобства
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': importances})
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# Вывод важности признаков
print(feature_importances)

# Формирование списка answer_list
answer_list = feature_importances.index.tolist()
print("answer_list =", answer_list)

     feature  importance
1  feature_1    0.317169
9  feature_9    0.186399
8  feature_8    0.131145
2  feature_2    0.065221
5  feature_5    0.056419
4  feature_4    0.056050
3  feature_3    0.052793
7  feature_7    0.051142
6  feature_6    0.046598
0  feature_0    0.037065
answer_list = [1, 9, 8, 2, 5, 4, 3, 7, 6, 0]


In [3]:
answer_list

[1, 9, 8, 2, 5, 4, 3, 7, 6, 0]

In [7]:
df = pl.read_csv('https://stepik.org/media/attachments/lesson/779915/fs_task2_22f.csv')

In [8]:
df

feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,target_reg,target_class
i64,f64,i64,i64,f64,i64,f64,f64,i64,i64,i64,i64,i64,f64,i64,f64,i64,i64,i64,f64,f64,str
0,4.101552,173,1,3.405865,88182,157.168032,1.9731e7,0,1,0,24,2015,191.996603,2020,0.1,0,174,9,4.76,57.15,"""engine_ignition"""
0,4.303506,173,1,-2.218952,86897,121.805143,1.5942e7,0,0,0,2,2015,193.547718,2015,0.1,1,174,7,4.26,60.82,"""engine_ignition"""
0,4.246379,174,1,-3.740321,44604,175.889164,2.1276e7,0,0,0,27,2013,177.922027,2019,0.1,1,174,6,5.98,46.51,"""engine_ignition"""
0,4.342874,172,1,3.139031,114610,151.653445,2.2504e7,0,0,0,49,2017,110.165268,2016,0.7,1,174,20,4.42,30.71,"""gear_stick"""
1,4.140287,148,0,-4.022916,114851,159.547387,1.1635e7,0,0,1,4,2017,190.915943,2020,0.1,0,174,25,4.28,59.18,"""engine_fuel"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,5.345345,173,1,-1.50418,42550,176.567014,1.2085e7,0,0,0,0,2013,109.83663,2018,0.1,1,174,8,1.92,31.75,"""break_bug"""
0,4.042874,172,1,-1.079727,65913,128.338875,1.6728e7,0,1,0,33,2014,196.975349,2016,0.1,0,174,16,3.84,50.87,"""engine_overheat"""
0,3.977816,174,1,-3.00148,62873,124.79559,1.8240e7,0,1,0,44,2014,179.247675,2016,0.1,0,174,14,4.04,54.3,"""engine_ignition"""


In [15]:
import polars as pl
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import LabelEncoder

# Загрузка данных
df 

# Подготовка данных
X = df.drop("target").to_pandas()
y = df["target"].to_numpy()

# 1. Проверка корреляций с целевой переменной
correlations = {}
for col in X.columns:
    correlations[col] = abs(np.corrcoef(X[col], y)[0,1])

# 2. Permutation Importance
rf = RandomForestClassifier(random_state=42)
rf.fit(X, y)

result = permutation_importance(rf, X, y, n_repeats=10, random_state=42)
importances = pd.Series(result.importances_mean, index=X.columns)

# 3. Поиск признаков с наименьшей важностью
potential_random = set()

# Добавляем признаки с очень низкой корреляцией
corr_threshold = 0.05
for col, corr in correlations.items():
    if abs(corr) < corr_threshold:
        potential_random.add(col)

# Добавляем признаки с очень низкой важностью
imp_threshold = np.percentile(importances, 10)
for col, imp in importances.items():
    if imp < imp_threshold:
        potential_random.add(col)

# Преобразуем в список для ответа
answer = list(potential_random)[:2]
print("answer =", answer)

KeyError: "['target'] not found in axis"